# Retries with Reflection

Sometimes Agents will not succeed in their task. This might be e.g. if their reasoning was wrong (i.e., they made the wrong plan) or inefficient (i.e., they are taking too long to get to an answer). In this case, we can use the re-try method Reflection (from https://arxiv.org/abs/2303.11366).

With Reflection, the Agent will think about why their plan failed and what it can do better next time. This reflection, along with the previous trial(s) will be available to the Agent in the next trial. This enables the Agent to solve harder problems.

This notebook will shows you how to use Reflection.

**Caution:** Reflection can cause your context size to blow up, because previous trials are appended to the prompt. With each re-trial, the prompt increases in size. If you encounter this issue, lower `AgentExecutor.max_trials` or `Reflector.max_iterations_per_trial`

First, we import libraries and load an LLM and tools; just like we would for an Agent without Reflection.

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "..."

In [2]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.agents.reflexion.react import ReactReflector
from langchain.llms import OpenAI

In [3]:
llm = OpenAI(temperature=0)

In [4]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

Next, we create a `Reflector` object. The default implementation is `ReactReflector`.

In [5]:
reflector = ReactReflector.from_llm(
    llm=llm,
    max_iterations_per_trial=2
)

Reflection only kicks in, wenn the Reflector determines that the current trial failed. For this, we have to pass in conditions that specify when a trial failed.
These can be:
- `max_iterations_per_trial`: how many steps can the Agent take in a trial?
- `max_execution_time_per_trial`: how much take can the Agent use in a trial?
- `max_action_repetition`: how often is the Agent allowed to suggest the exact same Actions?

Finally, let's initialize an agent with the `tools`, the `llm`, the type of `Agent` we want to use and our `Reflector`.

In [6]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    reflector=reflector,
    max_trials=3,
    verbose=True
)

Now let's test it out!

In [ ]:
question = "At what age did the main character of One Piece set sail to find the One Piece?"
agent.run(question)